In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder


from tensorflow.keras import layers


In [1]:
#linking to the driver
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#**data**

In [27]:

data = {"intents": [
    {"tag": "greeting",
     "patterns": ["Hi", "Hey", "Is anyone there?", "Hello", "Hay"],
     "responses": ["Hello", "Hi", "Hi there"]
    },
    {"tag": "goodbye",
     "patterns": ["Bye", "See you later", "Goodbye"],
     "responses": ["See you later", "Have a nice day", "Bye! Come back again"]
    },
    {"tag": "thanks",
     "patterns": ["Thanks", "Thank you", "That's helpful", "Thanks for the help","That's really helpful"],
     "responses": ["Happy to help!", "we happy to serve you", "You're most welcome!"]
    },
    {"tag": "about",
     "patterns": ["Who are you?", "What are you?", "Who you are?" ],
     "responses": ["I am Good taste restaurant , your bot assistant", "I'm fleksa bot, an Artificial Intelligent bot"]
    },
    {"tag": "name",
    "patterns": ["what is your name", "what should I call you", "whats your name?"],
    "responses": ["You can call  Good taste restaurant.", "I'm  Good taste restaurant!", "Just call me as  Good taste restaurant"]
    },
    {"tag": "help",
    "patterns": ["Could you help me?", "give me a hand please", "Can you help?", "What can you do for me?", "I need a support", "I need a help", "support me please"],
    "responses": ["Tell me how can assist you", "Tell me your problem to assist you", "Yes Sure, How can I support you"]
    },
    {"tag": "order",
    "patterns": ["I need to make an order", "i am hungery ", "I want to make an order", "can you create an account for me", "i want to eat","i want to order","order","i would like to make an order","order food","order a meal","i want to buy food"],
    "responses": ["here is the menue", "we have a diligous food ... menue :"]
    },
    {"tag": "delivery",
    "patterns": ["I need to make an order delivery ", "I want to eat at home", "can u deliver the order ?", "you have a delivery","i want food delivery","delivery","deliver me please"],
    "responses": ["ok here is the menue ", "we are happy to deliver to you "]
    },
    {"tag": "reservation",
    "patterns": ["I need to make a reservation ", "i want to reserve a table", "i want to book a table", "i would like to make a reservation","reservation","reserve a table","reserve me please","book a table","booking","i want to reserve"],
    "responses": [ "yes of course"]
    },
    {"tag": "complaint",
    "patterns": ["have a complaint", "I want to raise a complaint", "there is a complaint about a service"],
    "responses": ["Please provide us your complaint in order to assist you", "Please mention your complaint, we will reach you and sorry for any inconvenience caused"]
    },
    {"tag": "check out",
    "patterns": ["how much i owe you ", "how much money", "the check please", "i want to leave please ","that's it"],
    "responses": ["here is your check ", "check:"]
    },

]
}

#**preprocissing data**

In [28]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [29]:

lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [30]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index 
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

#**bulding and training models**

In [33]:
#model 1 using NN
modelnn = Sequential()
modelnn.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
modelnn.add(GlobalAveragePooling1D())
modelnn.add(Dense(16, activation='relu'))
modelnn.add(Dense(16, activation='relu'))
modelnn.add(Dense(num_classes, activation='softmax'))

modelnn.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

modelnn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_3   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_11 (Dense)            (None, 16)                272       
                                                                 
 dense_12 (Dense)            (None, 16)                272       
                                                                 
 dense_13 (Dense)            (None, 11)                187       
                                                                 
Total params: 16,731
Trainable params: 16,731
Non-trainable params: 0
__________________________________________________

In [8]:
#model 2 using LSTM
hidden_size =132
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(layers.LSTM(hidden_size, return_sequences=False,dropout=0.5))
model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 16)            16000     
                                                                 
 lstm_1 (LSTM)               (None, 132)               78672     
                                                                 
 dense_4 (Dense)             (None, 11)                1463      
                                                                 
Total params: 96,135
Trainable params: 96,135
Non-trainable params: 0
_________________________________________________________________


In [34]:

epochs = 350
history = modelnn.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/350
2/2 [==============================] - 0s 9ms/step - loss: 2.3957 - accuracy: 0.0806
Epoch 2/350
2/2 [==============================] - 0s 8ms/step - loss: 2.3938 - accuracy: 0.0968
Epoch 3/350
2/2 [==============================] - 0s 8ms/step - loss: 2.3922 - accuracy: 0.1290
Epoch 4/350
2/2 [==============================] - 0s 8ms/step - loss: 2.3903 - accuracy: 0.1774
Epoch 5/350
2/2 [==============================] - 0s 8ms/step - loss: 2.3889 - accuracy: 0.1774
Epoch 6/350
2/2 [==============================] - 0s 8ms/step - loss: 2.3871 - accuracy: 0.1774
Epoch 7/350
2/2 [==============================] - 0s 9ms/step - loss: 2.3854 - accuracy: 0.1774
Epoch 8/350
2/2 [==============================] - 0s 8ms/step - loss: 2.3837 - accuracy: 0.1774
Epoch 9/350
2/2 [==============================] - 0s 7ms/step - loss: 2.3819 - accuracy: 0.1774
Epoch 10/350
2/2 [==============================] - 0s 8ms/step - loss: 2.3799 - accuracy: 0.1774
Epoch 11/350
2/2 [===========

# **saving and loading model wieghts for LSTM arch**

In [20]:
#saving  last checkpoints 
model.save_weights('/content/gdrive/MyDrive/restuarant_chat_bot/restuarant_chat_bot') 

In [10]:
#loading checkpoints 
checkpoint_dir = '/content/gdrive/MyDrive/restuarant_chat_bot/restuarant_chat_bot'
model.load_weights(checkpoint_dir)

#**saving and loading model wieghts for NN arch**

In [35]:
#saving  last checkpoints 
model.save_weights('/content/gdrive/MyDrive/restuarant_chat_bot/NN/restuarant_chat_bot') 

In [ ]:
#loading checkpoints 
checkpoint_dir = '/content/gdrive/MyDrive/restuarant_chat_bot/NN/restuarant_chat_bot'
model.load_weights(checkpoint_dir)

#**menu**

In [11]:
#sample of the menu of good taste restuarant
menu= {
    "Chicken Thali":19.40,
    "Mango Lamb" : 8.50,
    "Chicken Dal" :8.50,
    "Chicken Korma" : 8.50,
    "Chili Chicken" :8.50

}

#**handling different situations**

In [36]:
#main function  
!pip install colorama     
def main (address='', phone_number=None,day="",time=None,number_of_chairs=0,name="") :
  customer_order=[]

  if number_of_chairs == 0 :
    # for ordering and delivery situation
    print ("menu : ")
    for i,j in menu.items() : # printing menu
      print(i,j)

      #taking the order
    order = input(Fore.GREEN + "ChatBot: " + Style.RESET_ALL + "please pick your order " )
    customer_order.append(order)

    while(True):
        temp = input (Fore.GREEN + "ChatBot: " + Style.RESET_ALL+ " anything else? y for more , n for checkout ")
        if temp == "n" :
          break
        else :
          temp = input (Fore.GREEN + "ChatBot: " + Style.RESET_ALL + " what else do you wish for ")
          customer_order.append(temp)
              
    confirmation = input (Fore.GREEN + "ChatBot: " + Style.RESET_ALL + f" if you wanna confirm the {customer_order} type 'y' " )
          
    if confirmation == "y": #for confirming the order
      #check 
      check = 0 
      try: # handling order out of the menu 
        for i in customer_order :
          check += menu[i]
        print(Fore.GREEN + "ChatBot: " + Style.RESET_ALL+" here is your check : " )
        print (check)
        print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice ((data['intents'][2])["responses"]))
      except : print ("sorry your order is not in the menu")
      
    else : #for not confirming the order
      print("okay bye")        

  else: # for reservation situation
    confirmation = input (Fore.GREEN + "ChatBot: " + Style.RESET_ALL + f" if you wanna confirm the reservation on : {day} at {time} o'clock with {number_of_chairs} chairs type 'y' " )
    if confirmation =='y' : 
      print(Fore.GREEN + "ChatBot: " + Style.RESET_ALL + " reservation confirmed ",np.random.choice ((data['intents'][2])["responses"]) ) 
    else :
      print("okay bye")   

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
import colorama 
colorama.init()
from colorama import Fore, Style, Back
import random
import pickle


def chat():
    # parameters
    max_len = 20 
      
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
          break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print (Fore.GREEN + "ChatBot: " + Style.RESET_ALL , np.random.choice(i['responses']))


        #handiling order situation
        if tag =="order" :
          main()
          break

        #handiling delivery situation
        if tag =="delivery" :
          name = input (Fore.GREEN + "ChatBot: " + Style.RESET_ALL + "what is your name sir ?")
          address = input (Fore.GREEN + "ChatBot: " + Style.RESET_ALL + "please enter your address sir ")
          phone_number = input (Fore.GREEN + "ChatBot: " + Style.RESET_ALL+ " please enter your phone number  ") 
          main(address=address,phone_number= phone_number,name=name )
          break
        
        #handing reservation situation

        if tag == "reservation" :
          day = input (Fore.GREEN + "ChatBot: " + Style.RESET_ALL+ " please enter day of reservation  ") 
          time = input (Fore.GREEN + "ChatBot: " + Style.RESET_ALL+ " please enter time of reservation  ") 
          number_of_chairs = input (Fore.GREEN + "ChatBot: " + Style.RESET_ALL+ " how many chairs do you need  ") 
          main (day=day,time=time,number_of_chairs= number_of_chairs )
          break

        

def user_interface() :
  print (Fore.GREEN + "hi, welcome to good taste restuarant, this is good taste chat bot, i can help you order food or reserve a table or even delivery (type quit to stop)!" + Style.RESET_ALL)
  chat()

#**user interface**

In [44]:
user_interface()

hi, welcome to good taste restuarant, this is good taste chat bot, i can help you order food or reserve a table or even delivery (type quit to stop)!
User: i want food delivery
ChatBot:  we are happy to deliver to you 
ChatBot: what is your name sir ?mustafa
ChatBot: please enter your address sir mansoura
ChatBot:  please enter your phone number  012001456
menu : 
Chicken Thali 19.4
Mango Lamb 8.5
Chicken Dal 8.5
Chicken Korma 8.5
Chili Chicken 8.5
ChatBot: please pick your order Mango Lamb
ChatBot:  anything else? y for more , n for checkout n
ChatBot:  if you wanna confirm the ['Mango Lamb'] type 'y' y
ChatBot:  here is your check : 
8.5
ChatBot: Happy to help!
